In [ ]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from datetime import datetime
import json

In [ ]:
# Set up for the scrape ##### Change to match yours
executable_path = {'executable_path': 'C:/Users/Joe/bin/chromedriver.exe'}

In [ ]:
StartTime= datetime.now()
fileName='DiscData.json'
with open(fileName,'a') as outfile:
    outfile.write("[")




with open(fileName,'a') as outfile:
    outfile.write("]")
print(f'The scrape took {datetime.now()-StartTime}')